Working with Neural Network Models

&copy; Hans Nieminen, Satakunta University of Applied Sciences

# Exercise 2.1

In [ ]:
import numpy as np

In [ ]:
def huber(y, y_hat, delta):
  a = y-y_hat
  return np.where(np.abs(a)<delta,                # np.where returns np.ndarray with same shape as a. For a single value x in a np.where produces a value based on the condition is np.abs(x)<delta:
                  0.5*a**2,                       # if condition is True, produce a value 0.5*x**2
                  delta*(np.abs(a) - 0.5*delta))  # if condition is False, produce a value delta*(np.abs(x) - 0.5*delta)

In [ ]:
y = np.array([2, 1.2, 1,2, 1.6, 0.9, 4, 6])
y_hat = np.array([1.8, 1.6, 0.4, 1.9, 1.7, 0.6, 1.2, 1.4])

In [ ]:
deltas = [1.0, 2.0, 3.0]

In [ ]:
results = []
for delta in deltas:
  loss = huber(y, y_hat, delta)
  results.append((delta,loss))

In [ ]:
for result in results:
  print('Huber with delta =',result[0],':', np.mean(result[1]).round(3))

Huber with delta = 1.0 : 0.842
Huber with delta = 2.0 : 1.392
Huber with delta = 3.0 : 1.694


# Exercise 2.2

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=425)

In [ ]:
model = SVC()

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly']
}

In [ ]:
folds = 4
score = 'accuracy'

In [ ]:
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=folds,
                           scoring=score,
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 48 candidates, totalling 192 fits


GridSearchCV(cv=4, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=1)

In [ ]:
print("Best Parameters: ", grid_search.best_params_)

Best Parameters:  {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}


# Exercise 2.3

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/haniemi/deeplearning/main/data/regression_gen_data.csv')
df.head()

,X1,X2,y
0,0.797402,1.727357,7.321581
1,0.585221,1.686770,6.690540
2,0.260030,0.741583,0.571638
3,0.258966,1.789537,6.380062
4,0.716532,1.462603,6.029785


In [ ]:
X = df.drop(columns=['y']).copy()
y = df['y']

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
# Linear regression
model1 = LinearRegression()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

In [ ]:
def search_hyperparameters(name, model, param_grid, X, y):
  gs = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
  gs.fit(X, y)
  print(name, gs.best_params_)
  return gs.best_params_

In [ ]:
# Ridge regression
param_grid = {
    'alpha': list(np.arange(0.01,1.0,0.01))}

params = search_hyperparameters('Ridge', Ridge(), param_grid, X_train, y_train)

model2 = Ridge(**params)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

Ridge {'alpha': 0.33}


In [ ]:
# Lasso regression
param_grid = {
    'alpha': list(np.arange(0.01,1.0,0.01))}

params = search_hyperparameters('Lasso', Lasso(), param_grid, X_train, y_train)

model3 = Lasso(**params)
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)

Lasso {'alpha': 0.01}


In [ ]:
# Elastic Net regression
param_grid = {
    'alpha': list(np.arange(0.01,1.0,0.01)),
    'l1_ratio': list(np.arange(0.1,1.0,0.1))}

params = search_hyperparameters('Elastic Net', ElasticNet(), param_grid, X_train, y_train)

model4 = ElasticNet(**params)
model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)

Elastic Net {'alpha': 0.01, 'l1_ratio': 0.30000000000000004}


In [ ]:
# Compare models performance with test data
tests = []
tests.append(('Linear regression', mean_squared_error(y_test, y_pred1)))
tests.append(('Ridge regression', mean_squared_error(y_test, y_pred2)))
tests.append(('Lasso regression', mean_squared_error(y_test, y_pred3)))
tests.append(('Elastic Net regression', mean_squared_error(y_test, y_pred4)))
print(tests)

[('Linear regression', 1.41229504092412), ('Ridge regression', 1.4380177699604093), ('Lasso regression', 1.4385441952024465), ('Elastic Net regression', 1.464475227639808)]
